In [1]:
import os

import folium

from exif import Image
import numpy as np
from tqdm import tqdm

In [9]:
scene = '0025'
# img_dir = f'/Volumes/Extreme_SSD/MegaDepth/scenes/{scene}/images/'
img_dir = f'../../data/scenes/{scene}/images/'

img_list = os.listdir(img_dir)

exif_list = []
for img in tqdm(img_list):

    try:
        img_path = os.path.join(img_dir, img)
        with open(img_path, 'rb') as src:
            img = Image(src)

        if img.has_exif and img.gps_longitude:
            exif_list.append(img_path)
    except:
        pass

len(exif_list)

  0%|          | 0/5386 [00:00<?, ?it/s]

100%|██████████| 5386/5386 [00:15<00:00, 340.50it/s]


434

In [ ]:
def decimal_coords(coords, ref):
 decimal_degrees = coords[0] + coords[1] / 60 + coords[2] / 3600
 if ref == "S" or ref == "W":
     decimal_degrees = -decimal_degrees
 return decimal_degrees

def image_coordinates(image_path):
    with open(image_path, 'rb') as src:
        img = Image(src)
    if img.has_exif:
        try:
            img.gps_longitude
            coords = (decimal_coords(img.gps_latitude,
                      img.gps_latitude_ref),
                      decimal_coords(img.gps_longitude,
                      img.gps_longitude_ref))
        except AttributeError:
            print('No Coordinates')
            coords = None
        except NotImplementedError:
            print('No Coordinates')
            coords = None
    else:
        print('The Image has no EXIF information')
        return None

    return coords

In [ ]:
# plot location on map
def plot_map(coords: list, paths: list):
    map = folium.Map(location=coords[0], zoom_start=25)
    for coord, path in zip(coords, paths):
        folium.Marker(coord, popup=path).add_to(map)
    return map

coords = [image_coordinates(img_path) for img_path in exif_list]
coords = [coord for coord in coords if coord is not None]
exif_list = [img_path for img_path, coord in zip(exif_list, coords) if coord is not None]
plot_map(coords, exif_list)

/var/folders/vn/89j8g3p55gb71ngxshtbg1r80000gn/T/ipykernel_75482/2441374891.py:14: RuntimeWarning: ASCII tag contains -1 fewer bytes than specified
  img.gps_latitude_ref),
/var/folders/vn/89j8g3p55gb71ngxshtbg1r80000gn/T/ipykernel_75482/2441374891.py:16: RuntimeWarning: ASCII tag contains -1 fewer bytes than specified
  img.gps_longitude_ref))


# Show camera poses

In [11]:
import pyproj

def gps_to_ecef_pyproj(coords):
    transformer = pyproj.Transformer.from_crs(
        {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},
        {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
    )
    x, y, z = transformer.transform(coords[0], coords[1], coords[2], radians=False)

    return x, y, z

def ecef_to_lla(coord):
    transformer = pyproj.Transformer.from_crs(
        {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
        {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},
    )
    lon, lat, alt = transformer.transform(coord[0], coord[1], coord[2], radians=False)
    return lon, lat, alt

# plot location on map
def plot_map(poses: list, points: list, color='red'):
    map = folium.Map(location=poses[0], zoom_start=30)
    
    # for coord in poses:
    #     folium.CircleMarker(coord, radius=1, color=color).add_to(map)

    for coord in points:
        folium.CircleMarker(coord, radius=0.1, color='blue').add_to(map)


    return map


In [15]:
import pycolmap

from megadepth.utils.projections import get_camera_poses

scene = '0022'
geo_model = pycolmap.Reconstruction(f"../../data/scenes/{scene}/geo-model")

poses = get_camera_poses(geo_model)
poses = [ecef_to_lla(pose) for pose in tqdm(poses)]

# for p in poses[:10]:
#     print(p)

poses = [p[:2][::-1] for p in poses]

p3d = geo_model.points3D
p3d = [p3d[p].xyz for p in p3d if p3d[p].track.length() > 20]
idx = np.random.choice(len(p3d), 5000)
p3d = [p3d[i] for i in idx]
p3d = [ecef_to_lla(p)[:2][::-1] for p in tqdm(p3d)]

plot_map(poses, p3d)

100%|██████████| 5000/5000 [00:05<00:00, 950.39it/s]
